In [15]:
import numpy as np
import pandas as pd
import random

In [16]:
%run Algorithms.ipynb

In [28]:
education_1_df = pd.read_csv('Datasets/campus-placement-prediction.csv')
education_2_df = pd.read_csv('Datasets/predict-dropout-or-academic-success.csv')
education_3_df = pd.read_csv('Datasets/student-performance-dataset.csv')

In [29]:
education_1_df = preprocessing(education_1_df)
education_2_df = preprocessing(education_2_df)
education_3_df = preprocessing(education_3_df)

sl_no
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gender          215 non-null    float64
 1   ssc_p           215 non-null    float64
 2   ssc_b           215 non-null    float64
 3   hsc_p           215 non-null    float64
 4   hsc_b           215 non-null    float64
 5   hsc_s           215 non-null    float64
 6   degree_p        215 non-null    float64
 7   degree_t        215 non-null    float64
 8   workex          215 non-null    float64
 9   etest_p         215 non-null    float64
 10  specialisation  215 non-null    float64
 11  mba_p           215 non-null    float64
 12  status          215 non-null    float64
dtypes: float64(13)
memory usage: 22.0 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Cou

In [30]:
def z_scores(df):
    used_cols = [col for col in df.columns if df[col].nunique() > 50]
    df_zscore = df[used_cols].apply(lambda col: (col - col.mean()) / col.std())
    
    return df_zscore

In [39]:
def modified_z_scores(df):
    used_cols = [col for col in df.columns if df[col].nunique() > 50]
    
    modified_z_scores = {}
    anomalies = {}
    
    for col in used_cols:
        median = df[col].median()
        mad = np.median(np.abs(df[col] - median))

        # Avoid division by zero
        if mad == 0:
            mad = 1e-9  # Small value to avoid division by zero
        
        # Calculate Modified Z-Scores
        mz_scores = 0.6745 * (df[col] - median) / mad

        modified_z_scores[col] = mz_scores
        
        # Detect anomalies where MZ > 3.5
        anomalies[col] = df[np.abs(mz_scores) > 3.5]
    
    return modified_z_scores, anomalies


mz_scores_finance_1, anomalies_finance_1 = modified_z_scores(education_1_df)

# Print anomalies
print("\nAnomalies detected in finance_1_df:")
for col, anomaly_data in anomalies_finance_1.items():
    print(f"Anomalies in column '{col}':")
    print(anomaly_data)


Anomalies detected in finance_1_df:
Anomalies in column 'ssc_p':
Empty DataFrame
Columns: [gender, ssc_p, ssc_b, hsc_p, hsc_b, hsc_s, degree_p, degree_t, workex, etest_p, specialisation, mba_p, status]
Index: []
Anomalies in column 'hsc_p':
     gender  ssc_p  ssc_b  hsc_p  hsc_b  hsc_s  degree_p  degree_t  workex  \
24      0.0   76.5    0.0   97.7    0.0    1.0     78.86       0.0     0.0   
177     1.0   73.0    1.0   97.0    0.0    0.0     79.00       1.0     1.0   

     etest_p  specialisation  mba_p  status  
24      97.4             1.0  74.01     0.0  
177     89.0             1.0  70.81     0.0  
Anomalies in column 'degree_p':
Empty DataFrame
Columns: [gender, ssc_p, ssc_b, hsc_p, hsc_b, hsc_s, degree_p, degree_t, workex, etest_p, specialisation, mba_p, status]
Index: []
Anomalies in column 'etest_p':
Empty DataFrame
Columns: [gender, ssc_p, ssc_b, hsc_p, hsc_b, hsc_s, degree_p, degree_t, workex, etest_p, specialisation, mba_p, status]
Index: []
Anomalies in column 'mba_p':

In [33]:
def iqr(df):
    used_cols = [col for col in df.columns if df[col].nunique() > 50]
    for col in used_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        
        # Calculate IQR
        IQR = Q3 - Q1
        
        # Calculate lower and upper bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

In [32]:
education_1_zscore = calculate_z_scores(education_1_df)
education_2_zscore = calculate_z_scores(education_2_df)
education_3_zscore = calculate_z_scores(education_3_df)

print(education_1_zscore.head())

['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p']
['Previous qualification (grade)', 'Admission grade', 'Curricular units 1st sem (grade)', 'Curricular units 2nd sem (grade)']
['StudyTimeWeekly', 'GPA']
      ssc_p     hsc_p  degree_p   etest_p     mba_p
0 -0.028022  2.263530 -1.137448 -1.288085 -0.596255
1  1.110776  1.100879  1.509743  1.084626  0.686019
2 -0.212741  0.152956 -0.322091  0.218398 -0.767682
3 -1.043981 -1.315270 -1.952804 -0.459519 -0.488256
4  1.708345  0.666835  0.941712  1.860464 -1.161965
